# Cleaning Variables

# TO-DO:
    - more years of data where it exists ( as far back as possible ) 
        - all variable done, except secondary counselors (which we might not even use) 
    - intentions dataframe -> sum up the Olympic and Garinger schools into one row each 
    - get high school graduates by ethnicity? 

In [539]:
import pandas as pd
import numpy as np

In [540]:
dataset = 'Common Core Data'
variable= 'Counselor'
filepath = dataset + '\\' + variable + '\\'
years = [2016, 2017, 2018, 2019, 2020, 2021]

counselor_df = pd.DataFrame()

for x in years:
    # print(x)
    if x == 2016:
        sheet = pd.read_csv(filepath + 'ccd_counselors_' + str(x) +'.csv')

        sheet.insert(0, 'year', x)

        sheet = sheet[sheet['LEA_NAME'] == 'Charlotte-Mecklenburg Schools'][['year','SECGUI']].reset_index(drop=True)
    else:
        sheet = pd.read_csv(filepath + 'ccd_counselors_' + str(2018) +'.csv')

        sheet.insert(0, 'year', x)

        sheet['SECGUI'] = sheet['STAFF_COUNT']

        sheet = sheet[ (sheet['LEA_NAME'] == 'Charlotte-Mecklenburg Schools') & (sheet['STAFF'] == 'Secondary School Counselors')][['year', 'SECGUI']].reset_index(drop=True)
        

    counselor_df = pd.concat([counselor_df, sheet])

In [541]:
counselor_df

,year,SECGUI
0,2016,206.23
0,2017,225.25
0,2018,225.25
0,2019,225.25
0,2020,225.25
0,2021,225.25


In [542]:
dataset = 'NC Stats Data'
variable = 'schools'
filepath = dataset + '\\' + variable + '\\'


schools_df = pd.read_csv(
    filepath + 'Schools_currentlyopen.csv', 
    header=0, 
    names=['agency_code','school', 'zip_code'], 
    usecols=[0, 1, 4],  
    dtype={'agency_code': 'object', 'zip_code' : str})

In [543]:
schools_df.head(5)

,agency_code,school,zip_code
0,600567,Char Eng Early College-UNCC,28262
1,600569,Charlotte Teacher Early College,28262
2,600302,Ardrey Kell High School,28277
3,600312,William Amos Hough High,28031
4,600361,Butler High School,28105


In [544]:
variable = 'intentions'
filepath = dataset + '\\' + variable + '\\'

intentions_df = pd.DataFrame()
years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]


for x in years:


    ## open file    
    ## read as data frame
    sheet = pd.read_excel(
        filepath + 'intentions_' + str(x) + '.xlsx' , index_col = None, #usecols=[0,2,3,4,5,6,7,8],
        skiprows = [0, 1, 2],
        header=0, 
        dtype={'School No':str}
    )

    total_graduates = int(sheet[ sheet['School Name'] == 'Total']['Total'])
    types_dict = {
        'Public Senior Institutions': int,
        'Private Senior Institutions': int, 
        'Community Technical College': int,
        'Private Junior Institutions': int, 
        'Trade Business Nursing': int, 
        'Other': int,
        'Total': int
        }


    rows_to_drop = list(sheet[ (sheet['School Name'].isna()) | (sheet['School Name'] == 'Total')].index)
    sheet.drop(rows_to_drop, inplace=True)


    sheet = sheet.astype(int, errors='ignore')
    sheet.replace({'*':2.5, np.nan:0}, inplace=True)
    sheet.loc[:,'Public Senior Institutions' :'Other'] = sheet.loc[:,'Public Senior Institutions':'Other'].div(sheet['Total'], axis=0)

    sheet.insert(0, 'year', x)
    intentions_df = pd.concat([intentions_df, sheet])


## reset index
intentions_df.reset_index(drop=True, inplace=True)

## renmaing now for joining later
intentions_df.rename(columns={'School No':'agency_code', 'School Name':'school'}, inplace=True)


## some unused code, to sum up olympic schools if desired
## will get updated to use agency codes

## t[ t['School Name'].str.contains('Olympic')][['Public Senior Institutions', 'Private Senior Institutions', 'Community Technical College', 
##        'Private Junior Institutions', 'Trade Business Nursing', 'Other', 'Total']]

In [545]:
intentions_df

,year,agency_code,school,Public Senior Institutions,Private Senior Institutions,Community Technical College,Private Junior Institutions,Trade Business Nursing,Other,Total
0,2011,600302,Ardrey Kell High,0.617312,0.123007,0.173121,0.005695,0.005695,0.079727,439.0
1,2011,600334,Cato Middle College High,0.696429,0.044643,0.142857,0.000000,0.044643,0.044643,56.0
2,2011,600361,Butler High,0.405063,0.088608,0.345389,0.009042,0.028933,0.122966,553.0
3,2011,600364,MG Davis Military & Global Lead Academy,0.500000,0.250000,0.250000,0.000000,0.000000,0.250000,10.0
4,2011,600376,E E Waddell High,0.234177,0.044304,0.462025,0.015823,0.015823,0.227848,158.0
...,...,...,...,...,...,...,...,...,...,...
346,2021,600576,West Charlotte High School,0.263889,0.032407,0.370370,0.000000,0.000000,0.333333,216.0
347,2021,600579,West Mecklenburg High School,0.173516,0.059361,0.264840,0.000000,0.000000,0.502283,219.0
348,2021,600582,Williams Secondary Montessori,0.600000,0.200000,0.100000,0.000000,0.000000,0.100000,25.0
349,2021,600592,Julius L. Chambers High School,0.349241,0.028200,0.340564,0.005423,0.005423,0.275488,461.0


In [546]:
## getting only the percentage of graduates in each category

## use this if using demographic breakdown

# intentions_pivot = intentions_df[['year', 'Graduate Intention', 'Percent']]
# intentions_pivot = intentions_pivot.pivot(index='year', columns='Graduate Intention', values='Percent').reset_index()
# intentions_pivot

In [547]:
dataset = 'NC Report Card Data'
variable = 'AP Classes'
filepath = dataset + '\\' + variable + '\\'

## read excel file 
AP_df = pd.read_excel(filepath + 'NC_AP'  + '.xlsx' , index_col = None, header=0, usecols=[0, 1, 2, 3, 5])

## choose only Charlotte-Mecklenburg Schools 
#AP_df = AP_df[ AP_df['agency_code'].str.startswith('600') == True]

## choose only high schools
#AP_df = AP_df[ AP_df['category_code'] == 'H']

In [548]:
## old code

## highschool only codes
# agency_codes = list(AP_df['agency_code'].drop_duplicates())

In [549]:
dataset = 'NC Report Card Data'
variable = 'CTE Certifications'
filepath = dataset + '\\' + variable + '\\'

credentials_df = pd.read_excel(filepath + 'CTE_Credentials'  + '.xlsx' , index_col = None, header=0, usecols=[0, 1, 4])

#credentials_df = credentials_df.loc[ credentials_df['agency_code'].isin(agency_codes), : ]

In [550]:
creds_enroll_df = pd.read_excel(
    filepath + 'CTE_Enrollment' +'.xlsx', 
    index_col = None, 
    header=None, 
    skiprows=[0], 
    names=['year', 'agency_code', 'students_CTE_pct'], 
    usecols=[0,1,4]
    )
creds_enroll_df

,year,agency_code,students_CTE_pct
0,2018,00A000,6.64
1,2018,00B000,1.62
2,2018,010303,41.36
3,2018,010310,55.86
4,2018,010324,72.23
...,...,...,...
3697,2020,990LEA,59.27
3698,2020,995316,94.67
3699,2020,995324,27.66
3700,2020,995330,80.45


In [551]:
creds_merged_df = credentials_df.merge(creds_enroll_df, on=['year', 'agency_code'])
creds_merged_df[['cred_earned_pct', 'students_CTE_pct']] = creds_merged_df[['cred_earned_pct', 'students_CTE_pct']] / 100

In [552]:
creds_merged_df

,year,agency_code,cred_earned_pct,students_CTE_pct
0,2018,010324,0.28,0.7223
1,2018,010348,0.17,0.7005
2,2018,010360,0.25,0.7641
3,2018,010388,0.44,0.7847
4,2018,010396,0.28,0.8015
...,...,...,...,...
1512,2020,960LEA,NaN,0.6039
1513,2020,970LEA,NaN,0.6898
1514,2020,980LEA,NaN,0.6755
1515,2020,990LEA,NaN,0.5927


In [553]:
dataset = 'NC Report Card Data'
variable = 'College Enrollment'
filepath =  dataset + '\\' + variable + '\\'

college_df = pd.read_excel(filepath + 'college enrollment'  + '.xlsx' , index_col = None, header=0, usecols=[1, 2, 3, 4, 6])

#college_df = college_df.loc[ college_df['agency_code'].isin(agency_codes), : ]
college_df.rename( columns = {'graduation_year' : 'year'}, inplace=True)
college_df.drop(college_df[ college_df['Status'] == 'NOENRO' ].index, inplace=True)
college_df.replace({np.nan: 0}, inplace=True)


In [554]:
## this dictionary can be used to replace all the subgroup codes with full-text 

group_definitions = pd.read_excel(filepath + 'Subgroup_Definitions' + '.xlsx', index_col = None, header=None, names=['subgroup', 'Subgroup_Full'])
group_definitions['subgroup'] = group_definitions['subgroup'].str.strip()


In [555]:
college_df = college_df.merge(group_definitions, on='subgroup')

In [556]:
college_pivot = pd.pivot_table(college_df, index= ['year', 'agency_code'], columns='subgroup').reset_index()

college_pivot.columns = college_pivot.columns.to_flat_index()

college_pivot.columns = [
    'year', 
    'agency_code', 
    'enroll_All',
    'enroll_AmericanIndian',
    'enroll_Asian',
    'enroll_Black',
    'enroll_Disadvantaged',
    'enroll_EnglishLearners',
    'enroll_Female',
    'enroll_Hispanic',
    'enroll_Male', 
    'enroll_Twoormore',
    'enroll_PacificIslander',
    'enroll_Disabilities',
    'enroll_White']

In [557]:
college_pivot

,year,agency_code,enroll_All,enroll_AmericanIndian,enroll_Asian,enroll_Black,enroll_Disadvantaged,enroll_EnglishLearners,enroll_Female,enroll_Hispanic,enroll_Male,enroll_Twoormore,enroll_PacificIslander,enroll_Disabilities,enroll_White
0,2011,010303,0.5641,0.00,0.000000,0.692300,0.571400,0.000000,0.521700,0.166600,0.625000,0.00000,0.000000,0.950000,0.600000
1,2011,010324,0.7561,0.95,0.950000,0.720000,0.547100,0.750000,0.827200,0.571400,0.696900,0.95000,0.000000,0.480000,0.779000
2,2011,010348,0.5320,0.00,0.500000,0.587300,0.469800,0.200000,0.534000,0.395800,0.529400,0.95000,0.000000,0.277700,0.609700
3,2011,010360,0.4829,0.95,0.500000,0.595200,0.500000,0.117600,0.466600,0.333300,0.500000,0.00000,0.000000,0.142800,0.222200
4,2011,010388,0.6484,0.95,0.500000,0.777700,0.514700,0.250000,0.688400,0.440000,0.612900,0.66660,0.000000,0.333300,0.655300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,2019,997LEA,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,0.000000,NaN
5493,2019,998706,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
5494,2019,998719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000
5495,2019,998LEA,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,0.000000


## Merge all of it together

In [558]:
# big merge statement
main_df = schools_df.merge(
    intentions_df.drop('school', axis=1), 
    on='agency_code', 
    how='right').merge( ## right merge gets the schools that are no longer active, can use this for aggregating olympic and garinger school data
        AP_df, 
        on=['agency_code', 'year'],
        how='left').merge(
            creds_merged_df, 
            on=['year', 'agency_code'], 
            how='left').merge(
                college_pivot, 
                on=['year', 'agency_code'], how='left').merge(
                    counselor_df, 
                    on=['year'], 
                    how='left')

In [559]:
main_df['year'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
      dtype=int64)

In [560]:
main_df.columns

Index(['agency_code', 'school', 'zip_code', 'year',
       'Public Senior Institutions', 'Private Senior Institutions',
       'Community Technical College', 'Private Junior Institutions',
       'Trade Business Nursing', 'Other', 'Total', 'category_code',
       'pct_ap_pass', 'pct_ap_participation', 'cred_earned_pct',
       'students_CTE_pct', 'enroll_All', 'enroll_AmericanIndian',
       'enroll_Asian', 'enroll_Black', 'enroll_Disadvantaged',
       'enroll_EnglishLearners', 'enroll_Female', 'enroll_Hispanic',
       'enroll_Male', 'enroll_Twoormore', 'enroll_PacificIslander',
       'enroll_Disabilities', 'enroll_White', 'SECGUI'],
      dtype='object')

In [561]:
## these are schools that did not show up in CMS currently active schools

## Year '2002' refers to the '2001-2002' school year

## comment format is as follows

        ## name of school
        ## agency code of school
        ## reason it was not on the main list of open and active high schools
non_active_schools = main_df[main_df['school'].isna()]['agency_code'].unique()
non_active_schools

## DROPPED FROM DATA 
        ## Charlotte-Mecklenburg Academy / Lincoln Heights Academy
        ## agency code : 600461
        ## has grades below high school
## DROPPED FROM DATA 

## E E Waddell High
## agency code : 600 376 
## closed in 2011

## Northwest School of the Arts
## agency code : 600482
## Active school
     ## not sure why it isn't in active schools from NCDPI



## Garinger - Math and Science 
## agency code : 600690
## consolidated in 2011

## Leadership and Public Service High School at Garinger
## agency code : 600691
## conslidated in 2011

## Business and Finance High School at Garinger
## agency code : 600692
## consolidated in 2011

## New Technology High at Garinger
## agency code : 600698
## conslidated in 2011



## School of Math, Engineering, Technology & Science at Olympic
## agency code : 600694
## conslidated in 2019

## School of Biotechnology, Health & Public Admin. at Olympic
## agency code : 600697
## conslidated in 2019

## THESE CODE HAVE TWO SCHOOL NAMES ASSOCIATED

## Name of some schools within Olympic Highschool changed
## 2011-2014 
## 2015- 2019

## agency code : 600695
## International Studies and Global Economics at Olympic 
## School of Tech Entrepreneurship/Adv Manufacturing at Olympic
## conslidated in 2019

## Renaissance High School at Olympic 
## Renaissance School of Arts/Technology at Olympic 
## agency code : 600693
## conslidated in 2019

## International Business and Communication Studies at Olympic 
## School of Exe Leadership/Entrepreneurial Develop at Olympic
## agency code : 600696
## conslidated in 2019


array(['600376', '600461', '600482', '600690', '600691', '600692',
       '600693', '600694', '600695', '600696', '600697', '600698',
       '600699', '600341', '600439', '600582'], dtype=object)

In [562]:
main_df.columns

Index(['agency_code', 'school', 'zip_code', 'year',
       'Public Senior Institutions', 'Private Senior Institutions',
       'Community Technical College', 'Private Junior Institutions',
       'Trade Business Nursing', 'Other', 'Total', 'category_code',
       'pct_ap_pass', 'pct_ap_participation', 'cred_earned_pct',
       'students_CTE_pct', 'enroll_All', 'enroll_AmericanIndian',
       'enroll_Asian', 'enroll_Black', 'enroll_Disadvantaged',
       'enroll_EnglishLearners', 'enroll_Female', 'enroll_Hispanic',
       'enroll_Male', 'enroll_Twoormore', 'enroll_PacificIslander',
       'enroll_Disabilities', 'enroll_White', 'SECGUI'],
      dtype='object')

In [563]:
main_df.rename(
    columns={
        'pct_ap_pass':'AP_pass_pct',
        'pct_ap_participation':'AP_part_pct',
        'Trade Business Nursing':'int_trdbusnrs',
        'Other': 'int_other', 
        'Community Technical College':'int_commcoll', 
        'Private Junior Institutions':'int_privjr', 
        'Private Senior Institutions':'int_privsr',
        'Public Senior Institutions':'int_pubsr',
        'Total':'int_totalsecondary',
        'cred_earned_pct':'CTE_cred_pct',
        'students_CTE_pct':'CTE_enroll_pct',
        'SECGUI' :'secondary_counselors'  }, inplace=True)

In [564]:
main_df.drop('category_code', axis=1, inplace=True)

In [565]:
main_df.columns

Index(['agency_code', 'school', 'zip_code', 'year', 'int_pubsr', 'int_privsr',
       'int_commcoll', 'int_privjr', 'int_trdbusnrs', 'int_other',
       'int_totalsecondary', 'AP_pass_pct', 'AP_part_pct', 'CTE_cred_pct',
       'CTE_enroll_pct', 'enroll_All', 'enroll_AmericanIndian', 'enroll_Asian',
       'enroll_Black', 'enroll_Disadvantaged', 'enroll_EnglishLearners',
       'enroll_Female', 'enroll_Hispanic', 'enroll_Male', 'enroll_Twoormore',
       'enroll_PacificIslander', 'enroll_Disabilities', 'enroll_White',
       'secondary_counselors'],
      dtype='object')

In [566]:
zip_codes = main_df['zip_code'].unique()
zip_codes

zip_2019_median_household_income = {
    '28277': 109251, 
    '28105': 117852, 
    '28215': 50704, 
    '28212': 46688, 
    '28205': 50200, 
    '28208': 42570, 
    '28078': 82225, 
    '28227': 43990, 
    '28262': np.nan,
    '28269': 60423, 
    '28209': 99628, 
    '28270': 82270, 
    '28210': 87928,
    '28214': 65706, 
    '28031': 91151,
    '28216': 64442, 
    '28273': 67621}

In [567]:
main_df['2019_med_hh_inc'] = main_df['zip_code'].map(zip_2019_median_household_income)

# year = main_df.pop('year')

# main_df.insert(1, 'year', year)

median_household_income = main_df.pop('2019_med_hh_inc')

main_df.insert(4, '2019_med_hh_inc', median_household_income)

In [568]:
## charlotte-mecklenburg academy and turning point academy rows
## these are showing up somewhere in the merge statement
## it's due to the type of merges used. will clean this up
CMA_TP_index = main_df[ (main_df['agency_code'] == '600461') | (main_df['agency_code'] == '600439')].index
main_df.drop(CMA_TP_index, inplace=True)

In [569]:
main_df.to_csv('Full_Data.csv')

In [571]:
hypothesis_df = main_df[['agency_code', 'school', 'zip_code', 'year', '2019_med_hh_inc', 'int_pubsr', 'int_commcoll',
                         'int_trdbusnrs', 'AP_pass_pct', 'AP_part_pct',
                          'CTE_cred_pct', 'CTE_enroll_pct', 'enroll_Disadvantaged']]

In [572]:
hypothesis_df['AP_pass_pct'].describe()

count    195.000000
mean       0.405702
std        0.230132
min        0.050000
25%        0.205000
50%        0.410000
75%        0.594349
max        0.870000
Name: AP_pass_pct, dtype: float64

In [584]:
summary_stats = hypothesis_df.drop(columns=['agency_code', 'year', 'school', '2019_med_hh_inc', 'zip_code'], axis=1)
summary_stats = summary_stats.describe().drop(labels=['25%', '50%', '75%'])
summary_stats = summary_stats.append(hypothesis_df[summary_stats.columns].apply(lambda x: (x.isna().sum()/x.size)* 100), ignore_index=True)
summary_stats.index = ['count', 'mean', 'std', 'min', 'max', 'missing']

In [585]:
summary_stats

,int_pubsr,int_commcoll,int_trdbusnrs,AP_pass_pct,AP_part_pct,CTE_cred_pct,CTE_enroll_pct,enroll_Disadvantaged
count,337.000000,337.000000,337.000000,195.000000,195.000000,24.000000,78.000000,209.000000
mean,0.397628,0.350344,0.015562,0.405702,0.218506,0.142500,0.661013,0.377958
std,0.181311,0.161921,0.019922,0.230132,0.125211,0.139728,0.147958,0.218764
min,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.221200,0.000000
max,1.000000,1.000000,0.138889,0.870000,0.720000,0.540000,0.979900,0.918900
missing,0.000000,0.000000,0.000000,42.136499,42.136499,92.878338,76.854599,37.982196
